In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
indep.shape

(400, 3)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from sklearn.naive_bayes import BernoulliNB

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'binarize': [0.0, 0.5, 1.0]
}

grid = GridSearchCV(BernoulliNB(), param_grid, scoring='f1', cv=5, verbose=2, n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 2.0],
                         'binarize': [0.0, 0.5, 1.0]},
             scoring='f1', verbose=2)

In [15]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

C:\Users\Priyan\anaconda3\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Priyan\anaconda3\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Priyan\anaconda3\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 0.1, 'binarize': 0.0}: 0.49240100865535336


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[85  0]
 [49  0]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.63      1.00      0.78        85
           1       0.00      0.00      0.00        49

    accuracy                           0.63       134
   macro avg       0.32      0.50      0.39       134
weighted avg       0.40      0.63      0.49       134



In [19]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.49039615846338536)

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006375,0.002266,0.008423,0.003533,0.1,0.0,"{'alpha': 0.1, 'binarize': 0.0}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.006464,0.001151,0.007518,0.001711,0.1,0.5,"{'alpha': 0.1, 'binarize': 0.5}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.004563,0.001124,0.006559,0.000935,0.1,1.0,"{'alpha': 0.1, 'binarize': 1.0}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.004090,0.001174,0.006551,0.001500,0.5,0.0,"{'alpha': 0.5, 'binarize': 0.0}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.004271,0.000879,0.007297,0.001172,0.5,0.5,"{'alpha': 0.5, 'binarize': 0.5}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,0.004785,0.000869,0.007651,0.000591,0.5,1.0,"{'alpha': 0.5, 'binarize': 1.0}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,0.006029,0.002039,0.007266,0.001853,1.0,0.0,"{'alpha': 1.0, 'binarize': 0.0}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,0.004881,0.001411,0.006357,0.001294,1.0,0.5,"{'alpha': 1.0, 'binarize': 0.5}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,0.005202,0.000633,0.005350,0.000962,1.0,1.0,"{'alpha': 1.0, 'binarize': 1.0}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,0.004757,0.000459,0.005147,0.000686,2.0,0.0,"{'alpha': 2.0, 'binarize': 0.0}",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [23]:
age_input=float(input("Age:"))
salary_input=float(input("EstimatedSalary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 31
EstimatedSalary: 160000
Sex Male 0 or 1: 1


In [24]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[0]


C:\Users\Priyan\anaconda3\envs\aiml\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  warnings.warn(
